# Written By SasiVatsal on 06-02-2022

In [88]:
# importing libraries
import pandas as pd
import random
import auth_sp as authorization # this is the script we created earlier
import numpy as np
from numpy.linalg import norm

In [89]:
# reading dataset into s_dataframe
s_dataframe = pd.read_csv("valence_arousal_dataset.csv")
# printing to check if it is read crct or not
s_dataframe

,id,genre,track_name,artist_name,valence,energy
0,0KpfYajJVVGgQ32Dby7e9i,acoustic,"Hey, Soul Sister",Train,0.768,0.885
1,6Zewjs7at1sPyc03lUoAHB,acoustic,Save Your Scissors,City and Colour,0.567,0.647
2,4E6cwWJWZw2zWf7VFbH7wf,acoustic,Love Song,Sara Bareilles,0.573,0.786
3,0EYKD5N0mldDeVTCzCfzqr,acoustic,Thrift Shop,Tyler Ward,0.472,0.533
4,2ds9vlk6o6BRCc2FAkmOvX,acoustic,Beat It,Fall Out Boy,0.841,0.936
...,...,...,...,...,...,...
11146,6wyWHg3QwEbEloYVKjaHcp,world-music,West African Drum Music,Drums Of The World,0.793,0.889
11147,3ToPv6Qyen8b7jOGv7bnlr,world-music,Mi Gente,A.B. Quintanilla III,0.966,0.833
11148,0zwL0IxW2DWeXtfN8PmlzV,world-music,Imidiwan Win Sahara,Tinariwen,0.826,0.680
11149,6TNyaNuxpeslfyGUQAZ0MG,world-music,Chant to King Selassie I,Augustus Pablo,0.805,0.429


In [90]:
# creating a new coloumn in our dataframe with name mood_vec and combining 
# the values of both valence and mood into one single coloumn
# values.tolist() method in pandas convers given col values into a list, pretty handy feature
s_dataframe["mood_vec"] = s_dataframe[["valence", "energy"]].values.tolist()
# printing to check
s_dataframe["mood_vec"]

0        [0.768, 0.885]
1        [0.567, 0.647]
2        [0.573, 0.786]
3        [0.472, 0.533]
4        [0.841, 0.936]
              ...      
11146    [0.793, 0.889]
11147    [0.966, 0.833]
11148     [0.826, 0.68]
11149    [0.805, 0.429]
11150    [0.935, 0.239]
Name: mood_vec, Length: 11151, dtype: object

In [91]:
# after new col dataframe looks like this
s_dataframe

,id,genre,track_name,artist_name,valence,energy,mood_vec
0,0KpfYajJVVGgQ32Dby7e9i,acoustic,"Hey, Soul Sister",Train,0.768,0.885,"[0.768, 0.885]"
1,6Zewjs7at1sPyc03lUoAHB,acoustic,Save Your Scissors,City and Colour,0.567,0.647,"[0.567, 0.647]"
2,4E6cwWJWZw2zWf7VFbH7wf,acoustic,Love Song,Sara Bareilles,0.573,0.786,"[0.573, 0.786]"
3,0EYKD5N0mldDeVTCzCfzqr,acoustic,Thrift Shop,Tyler Ward,0.472,0.533,"[0.472, 0.533]"
4,2ds9vlk6o6BRCc2FAkmOvX,acoustic,Beat It,Fall Out Boy,0.841,0.936,"[0.841, 0.936]"
...,...,...,...,...,...,...,...
11146,6wyWHg3QwEbEloYVKjaHcp,world-music,West African Drum Music,Drums Of The World,0.793,0.889,"[0.793, 0.889]"
11147,3ToPv6Qyen8b7jOGv7bnlr,world-music,Mi Gente,A.B. Quintanilla III,0.966,0.833,"[0.966, 0.833]"
11148,0zwL0IxW2DWeXtfN8PmlzV,world-music,Imidiwan Win Sahara,Tinariwen,0.826,0.680,"[0.826, 0.68]"
11149,6TNyaNuxpeslfyGUQAZ0MG,world-music,Chant to King Selassie I,Augustus Pablo,0.805,0.429,"[0.805, 0.429]"


In [92]:
# taking input from the user
user_song = input("Paste song URL: ")
# we dont need the whole url, instead we just need only the id of the song
# we slice the input from 31st index to 53 index, from which we will get the track id
track_id = user_song[31:53]

Paste song URL: https://open.spotify.com/track/2ZRo7axmMPeSVUvDbGkJah?si=f7c643c9cb184a1e


In [93]:
# getting app token from spotify api using tk.request_client_token method in tekore
# Tekore is alibrary which provide client for the Spotify Web API for Python, complete 
# with all available endpoints and authentication methods, async support 
# and loads of additional features
sp = authorization.authorize()

In [94]:
# getting details of the song given by the user using app token from spotify api
# by this we can know the enery and valence of the song given by the user from 
# which we can search for songs with similar valence and energy
track_features = sp.track_audio_features(track_id)
# making an array of valence and enery using numpy 
track_moodvec = np.array([track_features.valence, track_features.energy])
# making copy of original dataframe
ref_df = s_dataframe
# creating a new coloumn nameed distance in copy dataframe
# and Computing distances to all reference tracks
ref_df["distances"] = ref_df["mood_vec"].apply(lambda x: norm(track_moodvec-np.array(x)))
# Sort distances from lowest to highest, the song with least distance is the most
# similar one, as the distance increases similiarity decreases
# this can be imagined as a graph, farther the point more the distance.
ref_df_sorted = ref_df.sort_values(by = "distances", ascending = True)
# If the input track is in the reference set, it will have a distance of 0, but should not be recommendet
ref_df_sorted = ref_df_sorted[ref_df_sorted["id"] != track_id]
# storing the top 5 results into the new dataframe
final_df = ref_df_sorted.iloc[:5]

In [95]:
# printing final df
final_df

,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
3275,6uFuOOOwdxRezNeRmF4eb1,emo,The No Seatbelt Song,Brand New,0.224,0.455,"[0.224, 0.455]",0.002236
7380,5XHzBBcFHIO3RWpO5O1ajc,new-age,Hymn of the Universe (from Love is the Only Pr...,John Richardson,0.219,0.458,"[0.219, 0.458]",0.006083
3351,17eJJpSWm9JWl4wYmbwixM,folk,O I Long to Feel Your Arms Around Me,Father John Misty,0.233,0.456,"[0.233, 0.456]",0.008062
10361,37miw1ff4zH0LDuFrBvZ4i,spanish,Sabes,Reik,0.217,0.460,"[0.217, 0.46]",0.008544
5456,13FKvx5K0sgbzJXy34xiHs,indian,Rab Ka Shukrana - Reprise,Pritam,0.234,0.462,"[0.234, 0.462]",0.010296


In [96]:
recommended_songs_list = final_df

In [97]:
recommended_songs_list

,id,genre,track_name,artist_name,valence,energy,mood_vec,distances
3275,6uFuOOOwdxRezNeRmF4eb1,emo,The No Seatbelt Song,Brand New,0.224,0.455,"[0.224, 0.455]",0.002236
7380,5XHzBBcFHIO3RWpO5O1ajc,new-age,Hymn of the Universe (from Love is the Only Pr...,John Richardson,0.219,0.458,"[0.219, 0.458]",0.006083
3351,17eJJpSWm9JWl4wYmbwixM,folk,O I Long to Feel Your Arms Around Me,Father John Misty,0.233,0.456,"[0.233, 0.456]",0.008062
10361,37miw1ff4zH0LDuFrBvZ4i,spanish,Sabes,Reik,0.217,0.460,"[0.217, 0.46]",0.008544
5456,13FKvx5K0sgbzJXy34xiHs,indian,Rab Ka Shukrana - Reprise,Pritam,0.234,0.462,"[0.234, 0.462]",0.010296


In [98]:
# making a list of artists from recommended songs
artist_name=[]
artist_name = recommended_songs_list["artist_name"].tolist()
artist_name

['Brand New', 'John Richardson', 'Father John Misty', 'Reik', 'Pritam']

In [99]:
# making a list of id's from recommended songs
id_list = []
id_list = recommended_songs_list["id"].tolist()
id_list

['6uFuOOOwdxRezNeRmF4eb1',
 '5XHzBBcFHIO3RWpO5O1ajc',
 '17eJJpSWm9JWl4wYmbwixM',
 '37miw1ff4zH0LDuFrBvZ4i',
 '13FKvx5K0sgbzJXy34xiHs']

In [100]:
# making a list of track names from recommended songs
track_list = []
track_list = recommended_songs_list["track_name"].tolist()
track_list

['The No Seatbelt Song',
 'Hymn of the Universe (from Love is the Only Prayer)',
 'O I Long to Feel Your Arms Around Me',
 'Sabes',
 'Rab Ka Shukrana - Reprise']

In [101]:
# making a list of spotify links from recommended songs
links_list=[]
for sid in id_list:
    link = f'https://open.spotify.com/track/{sid}?si=77d73328f792498e'.format(sid)
    links_list.append(link)
# printing name of the recommended song , artist along with the spotify link
for i in range(5):
    print(f"                {track_list[i]} by {artist_name[i]}\n")
    print("play from here:",links_list[i])
    print("\n\n")

                The No Seatbelt Song by Brand New

play from here: https://open.spotify.com/track/6uFuOOOwdxRezNeRmF4eb1?si=77d73328f792498e



                Hymn of the Universe (from Love is the Only Prayer) by John Richardson

play from here: https://open.spotify.com/track/5XHzBBcFHIO3RWpO5O1ajc?si=77d73328f792498e



                O I Long to Feel Your Arms Around Me by Father John Misty

play from here: https://open.spotify.com/track/17eJJpSWm9JWl4wYmbwixM?si=77d73328f792498e



                Sabes by Reik

play from here: https://open.spotify.com/track/37miw1ff4zH0LDuFrBvZ4i?si=77d73328f792498e



                Rab Ka Shukrana - Reprise by Pritam

play from here: https://open.spotify.com/track/13FKvx5K0sgbzJXy34xiHs?si=77d73328f792498e





# The End Bixchh